# Project Title
### Data Engineering Capstone Project

#### Project Summary
This Project aims to provide an analytical model for immigrations to United States.It'll provide insights needed as well as figures that help in decision making.

The project follows the follow steps:
* Step 1: Scope the Project and Gather Data
* Step 2: Explore and Assess the Data
* Step 3: Define the Data Model
* Step 4: Run ETL to Model the Data
* Step 5: Complete Project Write Up

In [34]:
# Do all imports and installs here
import pandas as pd
from pyspark.sql.types import StructType, StructField, DoubleType, StringType, IntegerType, DateType, TimestampType
from pyspark.sql import SparkSession
import datetime

Initiating spark session

In [35]:
spark = SparkSession.builder.\
config("spark.jars.packages","saurfang:spark-sas7bdat:2.0.0-s_2.11")\
.enableHiveSupport().getOrCreate()

Define the Functions that will be used

In [36]:
#This function will check if there exists an earlier load for the table or this is the first time
def check_table(table_path):
    try:
        song_table_df = spark.read.parquet(table_path)
        return 'TRUE'
    except:
        return 'FALSE'

Set the config parameters that will be used

In [37]:
landing_zone_path="./Landing Zone/"
dwh_model_path="./DWH Model/"

### Step 1: Scope the Project and Gather Data

#### Scope 
The Solution will provide analytical model.The Model is dimensional star schema form.This is otimum for Data warehouse solutions to avail best performance for the analytical queries that will run on top of the model.

The Solution is based on:
1.Landing the source files in the landing zone.
2.Read the data in spark tables (schema on read).
3.Transforming the data to the analytical model in the dimensional star schema form.

The data extraction , transformations & loading is done with Pyspark SQL. The data is transformed and written to parque files.

#### Describe and Gather Data 
The data sets consists of the below source files:
1."i94_apr16_sub.sas7bdat":
Holds the Immigration transactions along with all the facts and measures related to it (City immigrated from , US city immigrated to,arrival date,arrival port,immigration mode,departure date,birthdate,visa type,gender,airline used).
The files are extracted on monthly basis.

2."us-cities-demographics.csv":
Holds the US cities details (city, state,median age of citizens,male population , female population,total population, foreign-born citizens)

3."city.txt":
Holds the countries names and corresponding codes , extracted from "I94_SAS_Labels_Description.SAS" file

4."airport-codes_csv.csv":
Holds the airports all details.

5."transport_mode.txt":
Holds transport modes used by immigrants ('Air','Sea',etc...) ,extracted from "I94_SAS_Labels_Description.SAS" file

6."visa_type.txt":
Holds the different type of Visa types and corresponding codes,extracted from "I94_SAS_Labels_Description.SAS" file

### Step 2: Explore and Assess the Data
#### Explore the Data 
Reading the data ,Identify data quality issues, like missing values in keys and amndatory values, duplicate data, granularity,checking distinct values.

#### Cleaning Steps
Filtering the requires set of data only.
removing records having null values in mandatory ccolumns
setting the required granularity

Read Immigration data and load to staging area

In [38]:
stg_immigration_df =spark.read.format('com.github.saurfang.sas.spark').load('../../data/18-83510-I94-Data-2016/i94_apr16_sub.sas7bdat')

In [39]:
stg_immigration_df.limit(10).toPandas()

,cicid,i94yr,i94mon,i94cit,i94res,i94port,arrdate,i94mode,i94addr,depdate,...,entdepu,matflag,biryear,dtaddto,gender,insnum,airline,admnum,fltno,visatype
0,6.0,2016.0,4.0,692.0,692.0,XXX,20573.0,NaN,None,NaN,...,U,None,1979.0,10282016,None,None,None,1.897628e+09,None,B2
1,7.0,2016.0,4.0,254.0,276.0,ATL,20551.0,1.0,AL,NaN,...,Y,None,1991.0,D/S,M,None,None,3.736796e+09,00296,F1
2,15.0,2016.0,4.0,101.0,101.0,WAS,20545.0,1.0,MI,20691.0,...,None,M,1961.0,09302016,M,None,OS,6.666432e+08,93,B2
3,16.0,2016.0,4.0,101.0,101.0,NYC,20545.0,1.0,MA,20567.0,...,None,M,1988.0,09302016,None,None,AA,9.246846e+10,00199,B2
4,17.0,2016.0,4.0,101.0,101.0,NYC,20545.0,1.0,MA,20567.0,...,None,M,2012.0,09302016,None,None,AA,9.246846e+10,00199,B2
5,18.0,2016.0,4.0,101.0,101.0,NYC,20545.0,1.0,MI,20555.0,...,None,M,1959.0,09302016,None,None,AZ,9.247104e+10,00602,B1
6,19.0,2016.0,4.0,101.0,101.0,NYC,20545.0,1.0,NJ,20558.0,...,None,M,1953.0,09302016,None,None,AZ,9.247140e+10,00602,B2
7,20.0,2016.0,4.0,101.0,101.0,NYC,20545.0,1.0,NJ,20558.0,...,None,M,1959.0,09302016,None,None,AZ,9.247161e+10,00602,B2
8,21.0,2016.0,4.0,101.0,101.0,NYC,20545.0,1.0,NY,20553.0,...,None,M,1970.0,09302016,None,None,AZ,9.247080e+10,00602,B2
9,22.0,2016.0,4.0,101.0,101.0,NYC,20545.0,1.0,NY,20562.0,...,None,M,1968.0,09302016,None,None,AZ,9.247849e+10,00608,B1


In [40]:
stg_immigration_df.printSchema()

root
 |-- cicid: double (nullable = true)
 |-- i94yr: double (nullable = true)
 |-- i94mon: double (nullable = true)
 |-- i94cit: double (nullable = true)
 |-- i94res: double (nullable = true)
 |-- i94port: string (nullable = true)
 |-- arrdate: double (nullable = true)
 |-- i94mode: double (nullable = true)
 |-- i94addr: string (nullable = true)
 |-- depdate: double (nullable = true)
 |-- i94bir: double (nullable = true)
 |-- i94visa: double (nullable = true)
 |-- count: double (nullable = true)
 |-- dtadfile: string (nullable = true)
 |-- visapost: string (nullable = true)
 |-- occup: string (nullable = true)
 |-- entdepa: string (nullable = true)
 |-- entdepd: string (nullable = true)
 |-- entdepu: string (nullable = true)
 |-- matflag: string (nullable = true)
 |-- biryear: double (nullable = true)
 |-- dtaddto: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- insnum: string (nullable = true)
 |-- airline: string (nullable = true)
 |-- admnum: double (nullable = 

In [41]:
stg_immigration_df.createOrReplaceTempView("stg_immigration_transactions")

Read Airport Data

In [42]:
stg_airport_schema = StructType([
        StructField('ident', StringType()),
        StructField('type', StringType()),
        StructField('name', StringType()),
        StructField('elevation_ft', IntegerType()),
        StructField('continent', StringType()),
        StructField('iso_country', StringType()),
        StructField('iso_region', StringType()),
        StructField('municipality', StringType()),
        StructField('gps_code', StringType()),
        StructField('iata_code', StringType()),
        StructField('local_code', StringType()),
        StructField('coordinates', StringType())
    ])

In [43]:
stg_airport_df =spark.read.csv('{}/airport-codes_csv.csv'.format(landing_zone_path),header=True,schema=stg_airport_schema)

In [44]:
stg_airport_df.limit(10).toPandas()

,ident,type,name,elevation_ft,continent,iso_country,iso_region,municipality,gps_code,iata_code,local_code,coordinates
0,00A,heliport,Total Rf Heliport,11,NA,US,US-PA,Bensalem,00A,None,00A,"-74.93360137939453, 40.07080078125"
1,00AA,small_airport,Aero B Ranch Airport,3435,NA,US,US-KS,Leoti,00AA,None,00AA,"-101.473911, 38.704022"
2,00AK,small_airport,Lowell Field,450,NA,US,US-AK,Anchor Point,00AK,None,00AK,"-151.695999146, 59.94919968"
3,00AL,small_airport,Epps Airpark,820,NA,US,US-AL,Harvest,00AL,None,00AL,"-86.77030181884766, 34.86479949951172"
4,00AR,closed,Newport Hospital & Clinic Heliport,237,NA,US,US-AR,Newport,None,None,None,"-91.254898, 35.6087"
5,00AS,small_airport,Fulton Airport,1100,NA,US,US-OK,Alex,00AS,None,00AS,"-97.8180194, 34.9428028"
6,00AZ,small_airport,Cordes Airport,3810,NA,US,US-AZ,Cordes,00AZ,None,00AZ,"-112.16500091552734, 34.305599212646484"
7,00CA,small_airport,Goldstone /Gts/ Airport,3038,NA,US,US-CA,Barstow,00CA,None,00CA,"-116.888000488, 35.350498199499995"
8,00CL,small_airport,Williams Ag Airport,87,NA,US,US-CA,Biggs,00CL,None,00CL,"-121.763427, 39.427188"
9,00CN,heliport,Kitchen Creek Helibase Heliport,3350,NA,US,US-CA,Pine Valley,00CN,None,00CN,"-116.4597417, 32.7273736"


In [45]:
stg_airport_df.printSchema()

root
 |-- ident: string (nullable = true)
 |-- type: string (nullable = true)
 |-- name: string (nullable = true)
 |-- elevation_ft: integer (nullable = true)
 |-- continent: string (nullable = true)
 |-- iso_country: string (nullable = true)
 |-- iso_region: string (nullable = true)
 |-- municipality: string (nullable = true)
 |-- gps_code: string (nullable = true)
 |-- iata_code: string (nullable = true)
 |-- local_code: string (nullable = true)
 |-- coordinates: string (nullable = true)



In [46]:
stg_airport_df.createOrReplaceTempView("stg_airport_prep")

In [47]:
spark.sql('Drop table if exists stg_airport')

DataFrame[]

Prepare and clense Airport Table

In [48]:
spark.sql('create table stg_airport as SELECT name,type,elevation_ft,iso_region,municipality,gps_code,iata_code,local_code ,coordinates  \
          FROM stg_airport_prep where iso_country="US" and local_code is not null \
           ').toPandas()

""


In [49]:
spark.sql(' SELECT name,type,elevation_ft,iso_region,municipality,gps_code,iata_code,local_code,coordinates  \
          FROM stg_airport').toPandas()

,name,type,elevation_ft,iso_region,municipality,gps_code,iata_code,local_code,coordinates
0,Total Rf Heliport,heliport,11.0,US-PA,Bensalem,00A,None,00A,"-74.93360137939453, 40.07080078125"
1,Aero B Ranch Airport,small_airport,3435.0,US-KS,Leoti,00AA,None,00AA,"-101.473911, 38.704022"
2,Lowell Field,small_airport,450.0,US-AK,Anchor Point,00AK,None,00AK,"-151.695999146, 59.94919968"
3,Epps Airpark,small_airport,820.0,US-AL,Harvest,00AL,None,00AL,"-86.77030181884766, 34.86479949951172"
4,Fulton Airport,small_airport,1100.0,US-OK,Alex,00AS,None,00AS,"-97.8180194, 34.9428028"
5,Cordes Airport,small_airport,3810.0,US-AZ,Cordes,00AZ,None,00AZ,"-112.16500091552734, 34.305599212646484"
6,Goldstone /Gts/ Airport,small_airport,3038.0,US-CA,Barstow,00CA,None,00CA,"-116.888000488, 35.350498199499995"
7,Williams Ag Airport,small_airport,87.0,US-CA,Biggs,00CL,None,00CL,"-121.763427, 39.427188"
8,Kitchen Creek Helibase Heliport,heliport,3350.0,US-CA,Pine Valley,00CN,None,00CN,"-116.4597417, 32.7273736"
9,Grass Patch Airport,small_airport,53.0,US-FL,Bushnell,00FA,None,00FA,"-82.21900177001953, 28.64550018310547"


In [50]:
spark.sql(' SELECT count(*)  \
          FROM stg_airport').toPandas()

,count(1)
0,21236


Read US City Data

In [51]:
stg_us_city_schema = StructType([
        StructField('City', StringType()),
        StructField('State', StringType()),
        StructField('Median Age', DoubleType()),
        StructField('Male Population', IntegerType()),
        StructField('Female Population', IntegerType()),
        StructField('Total Population', IntegerType()),
        StructField('Number of Veterans', IntegerType()),
        StructField('Foreign-born', IntegerType()),
        StructField('Average Household Size', DoubleType()),
        StructField('State Code', StringType()),
        StructField('Race', StringType()),
        StructField('Count', IntegerType())
    ])

In [52]:
stg_us_city_df=spark.read.option("delimiter", ";").csv('{}/us-cities-demographics.csv'.format(landing_zone_path),header=True,schema=stg_us_city_schema)

In [53]:
stg_us_city_df.limit(10).toPandas()

,City,State,Median Age,Male Population,Female Population,Total Population,Number of Veterans,Foreign-born,Average Household Size,State Code,Race,Count
0,Silver Spring,Maryland,33.8,40601,41862,82463,1562,30908,2.60,MD,Hispanic or Latino,25924
1,Quincy,Massachusetts,41.0,44129,49500,93629,4147,32935,2.39,MA,White,58723
2,Hoover,Alabama,38.5,38040,46799,84839,4819,8229,2.58,AL,Asian,4759
3,Rancho Cucamonga,California,34.5,88127,87105,175232,5821,33878,3.18,CA,Black or African-American,24437
4,Newark,New Jersey,34.6,138040,143873,281913,5829,86253,2.73,NJ,White,76402
5,Peoria,Illinois,33.1,56229,62432,118661,6634,7517,2.40,IL,American Indian and Alaska Native,1343
6,Avondale,Arizona,29.1,38712,41971,80683,4815,8355,3.18,AZ,Black or African-American,11592
7,West Covina,California,39.8,51629,56860,108489,3800,37038,3.56,CA,Asian,32716
8,O'Fallon,Missouri,36.0,41762,43270,85032,5783,3269,2.77,MO,Hispanic or Latino,2583
9,High Point,North Carolina,35.5,51751,58077,109828,5204,16315,2.65,NC,Asian,11060


In [54]:
stg_us_city_df.printSchema()

root
 |-- City: string (nullable = true)
 |-- State: string (nullable = true)
 |-- Median Age: double (nullable = true)
 |-- Male Population: integer (nullable = true)
 |-- Female Population: integer (nullable = true)
 |-- Total Population: integer (nullable = true)
 |-- Number of Veterans: integer (nullable = true)
 |-- Foreign-born: integer (nullable = true)
 |-- Average Household Size: double (nullable = true)
 |-- State Code: string (nullable = true)
 |-- Race: string (nullable = true)
 |-- Count: integer (nullable = true)



In [55]:
stg_us_city_df.createOrReplaceTempView("stg_us_city_prep")

Prepare and clense City Data to be a dimension on the state granularity

In [56]:
spark.sql('drop table IF EXISTS stg_us_city')

DataFrame[]

In [57]:
spark.sql('create table stg_us_city as SELECT state ,state_code , SUM(male_population) as male_population ,sum(female_population) as female_population,\
          sum(total_population) as total_population,sum(number_of_veterans) as number_of_veterans , \
         sum(foreign_born) as foreign_born, sum(Count)  as count from(\
          SELECT  state ,`State Code` as state_code  ,`Male Population` as male_population ,`Female Population` as female_population,\
          `Total Population` as total_population  ,`Number of Veterans` as number_of_veterans , \
         `Foreign-born` as foreign_born, `Average Household Size` as average_household_size, sum(Count)  as count\
           FROM stg_us_city_prep   group by \
           state ,`State Code`  ,`Median Age` ,`Male Population` ,`Female Population` , `Total Population`  ,`Number of Veterans` ,\
            `Foreign-born` , `Average Household Size`)A \
          group by state ,state_code').toPandas()

""


In [58]:
spark.sql('SELECT  state,state_code,male_population,female_population,total_population,number_of_veterans,foreign_born,count from stg_us_city').toPandas()

,state,state_code,male_population,female_population,total_population,number_of_veterans,foreign_born,count
0,Kentucky,KY,452483,477394,929877,56025.0,66488.0,999456
1,Virginia,VA,1160474,1203148,2363622,229766.0,269254.0,2610431
2,Massachusetts,MA,975713,1039744,2015457,66076.0,520597.0,2321003
3,Oklahoma,OK,714573,734422,1448995,95468.0,151174.0,1729503
4,Arkansas,AR,286479,303400,589879,31704.0,62108.0,643597
5,North Dakota,ND,95235,94255,189490,10299.0,11492.0,198197
6,Mississippi,MS,112147,130536,242683,14792.0,4861.0,249185
7,New Jersey,NJ,705736,723172,1428908,30195.0,477028.0,1795916
8,Montana,MT,87707,93587,181294,13854.0,5977.0,196224
9,Oregon,OR,707443,729066,1436509,78948.0,185753.0,1665343


In [59]:
spark.sql('SELECT count(*) from stg_us_city').toPandas()

,count(1)
0,49


Read Cities file

In [60]:
stg_all_city_schema = StructType([
        StructField('city_cd', IntegerType()),
        StructField('city', StringType())
    ])

In [61]:
stg_all_city_df =spark.read.csv('{}/city.txt'.format(landing_zone_path),header=True,schema=stg_all_city_schema)

In [62]:
stg_all_city_df.limit(10).toPandas()

,city_cd,city
0,582,MEXICO Air Sea
1,236,AFGHANISTAN
2,101,ALBANIA
3,316,ALGERIA
4,102,ANDORRA
5,324,ANGOLA
6,529,ANGUILLA
7,518,ANTIGUA-BARBUDA
8,687,ARGENTINA
9,151,ARMENIA


In [63]:
stg_all_city_df.printSchema()

root
 |-- city_cd: integer (nullable = true)
 |-- city: string (nullable = true)



In [64]:
stg_all_city_df.createOrReplaceTempView("stg_all_city")

Read Visa type file

In [65]:
stg_visa_type_schema = StructType([
        StructField('visa_cd', IntegerType()),
        StructField('visa', StringType())
    ])

In [66]:
stg_visa_type_df =spark.read.csv('{}/visa_type.txt'.format(landing_zone_path),header=True,schema=stg_visa_type_schema)

In [67]:
stg_visa_type_df.limit(10).toPandas()

,visa_cd,visa
0,1,Business
1,2,Pleasure
2,3,Student


In [68]:
stg_visa_type_df.printSchema()

root
 |-- visa_cd: integer (nullable = true)
 |-- visa: string (nullable = true)



In [69]:
stg_visa_type_df.createOrReplaceTempView("stg_visa_type")

Read transportation Mode data

In [70]:
stg_trans_mode_schema = StructType([
        StructField('trans_mode_cd', IntegerType()),
        StructField('trans_mode', StringType())
    ])

In [71]:
stg_trans_mode_df =spark.read.csv('{}/transport_mode.txt'.format(landing_zone_path),header=True,schema=stg_trans_mode_schema)

In [72]:
stg_trans_mode_df.limit(10).toPandas()

,trans_mode_cd,trans_mode
0,1,Air
1,2,Sea
2,3,Land
3,9,Not Reported


In [73]:
stg_trans_mode_df.printSchema()

root
 |-- trans_mode_cd: integer (nullable = true)
 |-- trans_mode: string (nullable = true)



In [74]:
stg_trans_mode_df.createOrReplaceTempView("stg_trans_mode")

Read date data

In [76]:
spark.sql("Drop table if exists stg_date")

DataFrame[]

In [77]:
spark.sql(" create table stg_date as select date_add(cast(to_timestamp('01-01-1960', 'mm-dd-yyyy') as date),arrdate ) as dt from stg_immigration_transactions\
 union     select date_add(cast(to_timestamp('01-01-1960', 'mm-dd-yyyy') as date),depdate) as dt from stg_immigration_transactions")

DataFrame[]

Load the Data Model which is a star schema dimensional model consisting of Fact_Immigration surrounded by denormalized dimensions :
All_City_Dim ,
US_City_Dim ,
Airport_Dim ,
Visa_type_Dim ,
Date_Dim

Load All_City_Dim

In [78]:
print('Load Dim_All_City')

Load Dim_All_City


In [79]:
dim_all_city_path='{}/all_city_dim.parquet'

In [80]:
#check if the parquet file is loaded before or not
tab_exist_flg=check_table(dim_all_city_path)

In [81]:
tab_exist_flg

'FALSE'

In [93]:
#if loaded before then make left join to increment on the existing data else load the full delta
if tab_exist_flg=='TRUE':
    dim_all_city_df=spark.read.parquet(dim_all_city_path.format(dwh_model_path))
    dim_all_city_df.createOrReplaceTempView("dim_all_city")
    all_city_insrt_df=spark.sql('SELECT city_cd ,city from stg_all_city where city_cd not in (select city_cd from dim_all_city)')
    all_city_insrt_df.write.parquet(dim_all_city_path.format(dwh_model_path), 'append')
else:
    all_city_insrt_df=spark.sql('SELECT city_cd ,city from stg_all_city')
    all_city_insrt_df.write.parquet(dim_all_city_path.format(dwh_model_path), 'append')

In [85]:
df_spark=spark.read.parquet(dim_all_city_path.format(dwh_model_path))

In [86]:
df_spark.limit(10).toPandas()

,city_cd,city
0,582,MEXICO Air Sea
1,236,AFGHANISTAN
2,101,ALBANIA
3,316,ALGERIA
4,102,ANDORRA
5,324,ANGOLA
6,529,ANGUILLA
7,518,ANTIGUA-BARBUDA
8,687,ARGENTINA
9,151,ARMENIA


Load Dim_US_city data

In [87]:
print('Load Dim_US_City')

Load Dim_US_City


In [88]:
dim_us_city_path='{}/us_city_dim.parquet'

In [89]:
#check if the parquet file is loaded before or not
tab_exist_flg=check_table(dim_us_city_path.format(dwh_model_path))

In [90]:
tab_exist_flg

'TRUE'

In [92]:
#if loaded before then make left join to increment on the existing data else load the full delta
if tab_exist_flg=='TRUE':
    dim_us_city_df=spark.read.parquet(dim_us_city_path.format(dwh_model_path))
    dim_us_city_df.createOrReplaceTempView("dim_us_city")
    us_city_insrt_df=spark.sql('SELECT  state,state_code,male_population,female_population,total_population,number_of_veterans,foreign_born,count from stg_us_city \
    where state_code not in (select state_code from dim_us_city) ')
    us_city_insrt_df.write.parquet(dim_us_city_path.format(dwh_model_path), 'append')
else:
    us_city_insrt_df=spark.sql('SELECT  state,state_code,male_population,female_population,total_population,number_of_veterans,foreign_born,count from stg_us_city')
    us_city_insrt_df.write.parquet(dim_us_city_path.format(dwh_model_path), 'append')

In [95]:
df_spark=spark.read.parquet(dim_us_city_path.format(dwh_model_path))

In [96]:
df_spark.count()

49

Load Dim_airport data

In [97]:
print('Load Dim_airport')

Load Dim_airport


In [98]:
dim_airport_path='{}/airport_dim.parquet'

In [99]:
#check if the parquet file is loaded before or not
tab_exist_flg=check_table(dim_airport_path.format(dwh_model_path))

In [100]:
tab_exist_flg

'TRUE'

In [101]:
#if loaded before then make left join to increment on the existing data else load the full delta
if tab_exist_flg=='TRUE':
    dim_airport_df=spark.read.parquet(dim_airport_path.format(dwh_model_path))
    dim_airport_df.createOrReplaceTempView("dim_airport")
    airport_insrt_df=spark.sql('SELECT name,type,elevation_ft,iso_region,municipality,gps_code,iata_code,local_code,coordinates  \
          FROM stg_airport \
          where local_code not in (select local_code from dim_airport) ')
    airport_insrt_df.write.parquet(dim_airport_path.format(dwh_model_path), 'append')
else:
    airport_insrt_df=spark.sql('SELECT name,type,elevation_ft,iso_region,municipality,gps_code,iata_code,local_code,coordinates  \
          FROM stg_airport')
    airport_insrt_df.write.parquet(dim_airport_path.format(dwh_model_path), 'append')

In [102]:
df_spark=spark.read.parquet(dim_airport_path.format(dwh_model_path))

In [103]:
df_spark.count()

21236

Load Dim_visa_type data

In [104]:
print('Load Dim_visa_type')

Load Dim_visa_type


In [105]:
dim_visa_type_path='visa_type_dim.parquet'.format(dwh_model_path)

In [106]:
#check if the parquet file is loaded before or not
tab_exist_flg=check_table(dim_visa_type_path.format(dwh_model_path))

In [107]:
tab_exist_flg

'FALSE'

In [108]:
#if loaded before then make left join to increment on the existing data else load the full delta
if tab_exist_flg=='TRUE':
    dim_visa_type_df=spark.read.parquet(dim_visa_type_path.format(dwh_model_path))
    dim_visa_type_df.createOrReplaceTempView("dim_visa_type")
    visa_type_insrt_df=spark.sql('SELECT visa_cd,visa  \
          FROM stg_visa_type \
          where visa_cd not in (select visa_cd from dim_visa_type) ')
    visa_type_insrt_df.write.parquet(dim_visa_type_path.format(dwh_model_path), 'append')
else:
    visa_type_insrt_df=spark.sql('SELECT visa_cd, visa  \
          FROM stg_visa_type')
    visa_type_insrt_df.write.parquet(dim_visa_type_path.format(dwh_model_path), 'append')

In [109]:
df_spark=spark.read.parquet(dim_visa_type_path.format(dwh_model_path))

In [110]:
df_spark.count()

3

Load Dim_transp_mode

In [111]:
print('Load Dim_trans_mode')

Load Dim_trans_mode


In [112]:
dim_trans_mode_path='trans_mode_dim.parquet'.format(dwh_model_path)

In [113]:
#check if the parquet file is loaded before or not
tab_exist_flg=check_table(dim_trans_mode_path.format(dwh_model_path))

In [114]:
tab_exist_flg

'FALSE'

In [115]:
#if loaded before then make left join to increment on the existing data else load the full delta
if tab_exist_flg=='TRUE':
    dim_trans_mode_df=spark.read.parquet(dim_trans_mode_path.format(dwh_model_path))
    dim_trans_mode_df.createOrReplaceTempView("dim_trans_mode")
    trans_mode_insrt_df=spark.sql('SELECT trans_mode_cd,trans_mode  \
          FROM stg_trans_mode \
          where trans_mode_cd not in (select trans_mode_cd from dim_trans_mode) ')
    trans_mode_insrt_df.write.parquet(dim_trans_mode_path.format(dwh_model_path), 'append')
else:
    trans_mode_insrt_df=spark.sql('SELECT trans_mode_cd, trans_mode  \
          FROM stg_trans_mode')
    trans_mode_insrt_df.write.parquet(dim_trans_mode_path.format(dwh_model_path), 'append')

In [116]:
df_spark=spark.read.parquet(dim_trans_mode_path.format(dwh_model_path))

In [117]:
df_spark.count()

4

Load dim_date Table

In [118]:
print('Load Dim_date'.format(dwh_model_path))

Load Dim_date


In [119]:
dim_date_path='date_dim.parquet'.format(dwh_model_path)

In [120]:
#check if the parquet file is loaded before or not
tab_exist_flg=check_table(dim_date_path.format(dwh_model_path))

In [121]:
tab_exist_flg

'FALSE'

In [122]:
#if loaded before then make left join to increment on the existing data else load the full delta
if tab_exist_flg=='TRUE':
    dim_date_df=spark.read.parquet(dim_date_path.format(dwh_model_path))
    dim_date_df.createOrReplaceTempView("dim_date")
    dt_insrt_df=spark.sql('SELECT dt, day,week ,month ,year ,weekday\
          FROM stg_dt \
          where dt not in (select dt from dim_dt) ')
    date_insrt_df.write.parquet(dim_date_path.format(dwh_model_path), 'append')
else:
    date_insrt_df=spark.sql('SELECT dt, dayofmonth(dt) as day, weekofyear(dt) as week, \
                                                month(dt) as month, year(dt) as year, dayofweek(dt) as weekday  \
          FROM stg_date')
    date_insrt_df.write.parquet(dim_date_path.format(dwh_model_path), 'append')

In [123]:
df_spark=spark.read.parquet(dim_date_path.format(dwh_model_path))

In [124]:
df_spark.limit(10).toPandas()

,dt,day,week,month,year,weekday
0,2016-04-19,19,16,4,2016,3
1,2016-09-12,12,37,9,2016,2
2,2015-04-16,16,16,4,2015,5
3,2016-06-30,30,26,6,2016,5
4,2016-06-03,3,22,6,2016,6
5,2016-07-20,20,29,7,2016,4
6,2016-07-06,6,27,7,2016,4
7,2016-04-26,26,17,4,2016,3
8,2016-08-05,5,31,8,2016,6
9,2016-05-13,13,19,5,2016,6


In [125]:
df_spark.count()

236

Load The Fact_Immigration Table

In [126]:
print('Load Fact_Immigration Table')

Load Fact_Immigration Table


In [128]:
fact_immig_path='{}/immig_fact.parquet'

In [131]:
immig_insrt_df=spark.sql("select \
cast(cast(i94yr as integer) as string)||''||case when length(cast(cast(i94mon as integer) as string)) <2 then '0'||cast(cast(i94mon as integer) as string)\
else cast(cast(i94mon as integer) as string) end as  extract_month, \
cast(i94res as integer) as immig_from_city_cd, \
i94port as immig_port_cd ,\
date_add(cast(to_timestamp('01-01-1960', 'mm-dd-yyyy') as date),arrdate ) as arrival_date,\
cast(coalesce(i94mode,9) as integer) as immig_transport_mode ,\
i94addr as immig_to_city_cd ,\
date_add(cast(to_timestamp('01-01-1960', 'mm-dd-yyyy') as date),depdate) as depart_date ,\
cast(i94visa as integer) as visa_type_cd ,\
matflag as match_flg ,\
cast(biryear as integer) as resp_birth_year ,\
dtaddto as US_dt_addmt_to ,\
gender as rep_gender ,\
airline as airline_used ,\
admnum as admission_no , \
fltno as flight_no_used , \
visatype as visa_type \
from stg_immigration_transactions limit 1000000")
immig_insrt_df.write.parquet(fact_immig_path.format(dwh_model_path), 'append')

In [ ]:
# Performing cleaning tasks here





### Step 3: Define the Data Model
#### 3.1 Conceptual Data Model
Map out the conceptual data model and explain why you chose that model

#### 3.2 Mapping Out Data Pipelines
List the steps necessary to pipeline the data into the chosen data model

### Step 4: Run Pipelines to Model the Data 
#### 4.1 Create the data model
Build the data pipelines to create the data model.

In [ ]:
# Write code here

#### 4.2 Data Quality Checks
Explain the data quality checks you'll perform to ensure the pipeline ran as expected. These could include:
 * Integrity constraints on the relational database (e.g., unique key, data type, etc.)
 * Unit tests for the scripts to ensure they are doing the right thing
 * Source/Count checks to ensure completeness
 
Run Quality Checks

In [ ]:
# Perform quality checks here

#### 4.3 Data dictionary 
Create a data dictionary for your data model. For each field, provide a brief description of what the data is and where it came from. You can include the data dictionary in the notebook or in a separate file.

#### Step 5: Complete Project Write Up
* Clearly state the rationale for the choice of tools and technologies for the project.
* Propose how often the data should be updated and why.
* Write a description of how you would approach the problem differently under the following scenarios:
 * The data was increased by 100x.
 * The data populates a dashboard that must be updated on a daily basis by 7am every day.
 * The database needed to be accessed by 100+ people.